24.03.15. 과제 - MNIST FASHION => 클래스 만들어서 해보기

# 1. 모듈 로딩 및 데이터 준비

In [128]:
from sklearn.datasets import fetch_openml
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd

from torch.utils.data import Dataset, DataLoader

In [129]:
file = fetch_openml('Fashion-MNIST', parser = 'auto')
file.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

# 2. 데이터셋 생성

In [130]:
feature = file['data']
feature

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,119,114,130,76,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,22,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,33,96,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69996,0,0,0,0,0,0,0,0,0,31,...,0,0,0,0,0,0,0,0,0,0
69997,0,0,0,0,0,0,0,0,0,0,...,27,0,0,0,0,0,0,0,0,0
69998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [131]:
target = file['target']
target

0        9
1        0
2        0
3        3
4        0
        ..
69995    9
69996    1
69997    8
69998    1
69999    5
Name: class, Length: 70000, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

In [132]:
feature.isna().sum()

pixel1      0
pixel2      0
pixel3      0
pixel4      0
pixel5      0
           ..
pixel780    0
pixel781    0
pixel782    0
pixel783    0
pixel784    0
Length: 784, dtype: int64

In [133]:
feature.duplicated().sum()

0

In [134]:
feature = feature/256
feature

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.003906,0.0,0.0,0.000000,0.000000,...,0.464844,0.445312,0.507812,0.296875,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.085938,...,0.000000,0.000000,0.003906,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.128906,0.375000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
69996,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.121094,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
69997,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.105469,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
69998,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [135]:
# 사용자 정의 Dataset 클래스


class DLDataset(Dataset):
    def __init__(self, x_data, y_data):
        super().__init__()
        
        # x,y 데이터 ==> ndarray
        x_data = x_data.values if isinstance(x_data, pd.DataFrame) else x_data
        y_data = y_data.values if isinstance(y_data, pd.DataFrame) else y_data

        # ndarray ==> tensor        
        self.feature = torch.FloatTensor(x_data)
        self.target = torch.LongTensor(y_data)
        
    def __len__(self):
        return self.target.shape[0]
    
    def __getitem__(self, index):
        return self.feature[index], self.target[index]

In [136]:
print(f"Feature : {feature.shape}, {feature.ndim}D")
print(f"Target : {target.shape}, {target.ndim}D")

Feature : (70000, 784), 2D
Target : (70000,), 1D


In [137]:
# 전처리
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(target)

target = le.transform(target)
target = pd.Series(target)
target.unique()

array([9, 0, 3, 2, 7, 5, 1, 6, 4, 8])

In [138]:
# 데이터 셋 생성 
my_dataset = DLDataset(feature, target)

In [139]:
# 학습용, 검증용, 테스트용 Dataset 생성
from torch.utils.data import random_split

seed = torch.Generator().manual_seed(2)

trainDS, validDS, testDS = random_split(my_dataset, [0.6, 0.25, 0.15], generator = seed)

In [140]:
# 학습용, 검증용, 테스트용 DataLoader 생성

batch = 100

trainDL = DataLoader(trainDS, batch_size = batch)
validDL = DataLoader(validDS, batch_size = batch)
testDL = DataLoader(testDS, batch_size = batch)

In [141]:
# Epoch당 반복 단위
print(f"batch_size : {batch}")
print(f"trainDS : {len(trainDS)}개, validDS : {len(validDS)}개, testDS: {len(testDS)}개")
print(f"trainDL : {len(trainDL)}개, validDL : {len(validDL)}개, testDL: {len(testDL)}개")

batch_size : 100
trainDS : 42000개, validDS : 17500개, testDS: 10500개
trainDL : 420개, validDL : 175개, testDL: 105개


# 데이터셋과 데이터 로더는 뭐가 다르지? 임포트는 뭘 임포트 하고자 하는거지?

# 4. Model 클래스 정의 : 입출력 피처수, 층수, 은닉층의 노드 수

In [118]:
class CModel(nn.Module):
    def __init__(self, in_, out_):
        super().__init__()
        self.input_layer = nn.Linear(in_, 500)
        self.relu = nn.ReLU()
        self.hidden_layer_01 = nn.Linear(500, 400)
        self.hidden_layer_02 = nn.Linear(400, 300)
        self.hidden_layer_03 = nn.Linear(300, 200)
        self.hidden_layer_04 = nn.Linear(200, 100)
        self.hidden_layer_05 = nn.Linear(100, 50)
        self.output_layer = nn.Linear(50, out_)
        
    def forward(self, x):
        x = self.input_layer(x)
        x = self.relu(x)
        x = self.hidden_layer_01(x)
        x = self.relu(x)
        x = self.hidden_layer_02(x)
        x = self.relu(x)
        x = self.hidden_layer_03(x)
        x = self.relu(x)
        x = self.hidden_layer_04(x)
        x = self.relu(x)
        x = self.hidden_layer_05(x)
        x = self.relu(x)
        x = self.output_layer(x)
        
        return x

# 5. 학습 준비

In [119]:
# 실행 디바이스 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 학습횟수
EPOCHS = 30

In [120]:
# 모델 인스턴스
IN, OUT = my_dataset.feature.shape[1], 10
print(IN, OUT)

model = CModel(IN, OUT).to(DEVICE)

784 10


# 여기서 디바이스에 저장한다는 건 뭐지?

In [121]:
# 손실함수
loss_fx = nn.CrossEntropyLoss().to(DEVICE)

In [122]:
# 최적화 인스턴스
import torch.optim as optim
OPTIMIZER = optim.Adam(model.parameters())

# 여기서 model.parameters()는 뭐지?

In [123]:
import torchmetrics.functional as metrics

# 학습 진행 함수
# => 정규화, 경사하강법, 드랍아웃 등의 기능 활성화
def training ():
    model.train()              # =====================> 뭐지?????????????????
    
    train_loss = []
    
    for cnt, (feature, target) in enumerate(trainDL):
        # 배치 크기만큼의 학습 데이터 준비
        feature, target = feature.to(DEVICE), target.to(DEVICE)
        
        # 학습
        predict_target = model(feature)
        
        # 손실
        loss = loss_fx(predict_target, target)
        train_loss.append(loss)
        
        # W, b 업데이트
        OPTIMIZER.zero_grad()    # 0으로 만들기
        loss.backward()          # 역방향 학습
        OPTIMIZER.step()         # 업데이트
        
        # 정확도
        acc = metrics.accuracy(predict_target, target, task = 'multiclass', num_classes = 10)
        
    # 에포크 단위 학습 진행 출력
    # print(f"[Train Loss] {loss}")
    
    return train_loss

# 학습 함수에서 모르는거 있씀

In [124]:
# 검증 및 평가 진행 함수
# 매개변수 dataLoader : 검증 또는 테스트 데이터셋에 대한 Loader

def testing(dataLoader):
    model.eval()              # =====================> 뭐지?????????????????
    
    with torch.no_grad():      # =====================> 뭐지?????????????????
        # 배치 크기 만큼 학습 진행 및 저장
        val_loss = []
        for cnt, (feature, target) in enumerate(dataLoader):
            # 배치 크기 만큼의 학습 데이터 준비
            feature, target = feature.to(DEVICE), target.to(DEVICE)
            
            # 학습
            predict_target = model(feature)
            
            # 손실 계산
            loss = loss_fx(predict_target, target)
            val_loss.append(loss)
            
            
        # 에포크 단위 진행 메시지 출력
        # print(f"[Valid Loss] {loss}")
        
        return val_loss    

# 평가 함수에서 모르는거 이씀

In [125]:
# 예측 함수
def predict():
    pass

# 예측 함수 함 만들어보자

# 6. 학습 진행

In [126]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience = 10, verbose = True)

C:\ProgramData\anaconda3\envs\Torch_PY38\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


# 각 파라미터는 뭘 의미하는 거지?

In [127]:
for eps in range(EPOCHS):
    # 학습
    train_loss = training()
    
    # 검증
    valid_loss = testing(validDL)
    
    print(f"[{eps}/{EPOCHS}] \nTRAIN {sum(train_loss)}/{len(valid_loss)} \nVALID {sum(valid_loss)/len(valid_loss)}\n")

[0/30] 
TRAIN 307.5705261230469/175 
VALID 0.5283448696136475

[1/30] 
TRAIN 188.97361755371094/175 
VALID 0.40116333961486816

[2/30] 
TRAIN 164.08372497558594/175 
VALID 0.39781898260116577

[3/30] 
TRAIN 151.776611328125/175 
VALID 0.3822077810764313

[4/30] 
TRAIN 143.45399475097656/175 
VALID 0.35464218258857727

[5/30] 
TRAIN 135.76560974121094/175 
VALID 0.33647847175598145

[6/30] 
TRAIN 128.15318298339844/175 
VALID 0.34436121582984924

[7/30] 
TRAIN 123.51822662353516/175 
VALID 0.3423166871070862

[8/30] 
TRAIN 116.15044403076172/175 
VALID 0.35077035427093506

[9/30] 
TRAIN 114.29352569580078/175 
VALID 0.33195945620536804

[10/30] 
TRAIN 108.3534927368164/175 
VALID 0.32803842425346375

[11/30] 
TRAIN 104.0967025756836/175 
VALID 0.31767934560775757

[12/30] 
TRAIN 99.5213623046875/175 
VALID 0.34363147616386414

[13/30] 
TRAIN 98.22340393066406/175 
VALID 0.3477935791015625

[14/30] 
TRAIN 95.08396911621094/175 
VALID 0.3652873933315277

[15/30] 
TRAIN 89.86033630371094/1